## Проект: Анализ вакансий из HeadHunter

In [186]:
# Начинаем с импорта библиотек:
import psycopg2
import pandas as pd

In [187]:
# параметры подключения:

DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [277]:
# Создаём соединение с заданными параметрами:

connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

In [198]:
# код запроса представляет из себя строковую переменную
# n = 3
query_vacancies = f'''select *
            from vacancies
        '''

In [199]:
# EMPLOYERS_INDUSTRIES
query_employers_industries = f'''select *
            from employers_industries
        '''

In [ ]:
# INDUSTRIES
query_industries = f'''select *
            from industries
        '''
ind_df = pd.read_sql_query(query_industries, connection)

In [ ]:
# EMPLOYERS
query_employers = f'''select *
            from employers
        '''
emp_df = pd.read_sql_query(query_employers, connection)

In [ ]:
# AREAS
query_areas = f'''select *
            from areas
        '''
areas_df = pd.read_sql_query(query_areas, connection)

In [ ]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vacancies_df = pd.read_sql_query(query_vacancies, connection)
# vacancies_df

In [ ]:
ei_df = pd.read_sql_query(query_employers_industries, connection)

In [207]:
ei_df.head(3)

,employer_id,industry_id
0,2393,7.540
1,2393,7.539
2,72977,7.540


In [ ]:
vacancies_df.head(3)

# 3. Предварительный анализ данных

In [246]:
# Сколько вакансий есть в базе?
vacancies_df.shape[0]

49197

In [247]:
# Сколько работодателей?
emp_df['id'].shape[0]

23501

In [248]:
# Сколько регионов?
areas_df['id'].shape[0]

1362

In [270]:
# Сколько сфер деятельности?
print(ei_df['industry_id'].nunique())

294


# Вывод
по предварительному анализу данных
общее количество вакансий 49197, общее число работодателей составляет 23501, при этом, охвачено 1362 региона (в т.ч. городов) и 294 сферы деятельности.

# 4. Детальный анализ вакансий

In [253]:
# Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
# Выберите пятёрку лидеров по количеству вакансий:
query_vacancies_areas = f'''select 
                                a.name,
                                COUNT(v.id)                                
                            from vacancies v
                                JOIN areas a ON v.area_id = a.id
                            GROUP BY 1
                            ORDER BY 2 DESC
                            LIMIT 5
                         '''


In [255]:
vacancies_areas_df = pd.read_sql_query(query_vacancies_areas, connection)

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\2610693192.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_areas_df = pd.read_sql_query(query_vacancies_areas, connection)


In [256]:
vacancies_areas_df

,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


In [257]:
# Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?
mask1 = vacancies_df['salary_from'] > 0
mask2 = vacancies_df['salary_to'] > 0
display(vacancies_df[mask1 | mask2].shape[0])

24073

In [276]:
# Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.
print(vacancies_df['salary_from'].mean(), vacancies_df['salary_to'].mean())

71064.65790086107 110536.74192277384


In [259]:
# Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. 
# Какая пара находится на втором месте по популярности?
query_vacancies_sch_emp = f'''select 
                                v.schedule,
                                v.employment,
                                COUNT(v.id)                                
                            from vacancies v
                            GROUP BY 1,2
                            ORDER BY 3 DESC
                            LIMIT 5
                         '''

In [260]:
vacancies_sch_emp = pd.read_sql_query(query_vacancies_sch_emp, connection)

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\936489690.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_sch_emp = pd.read_sql_query(query_vacancies_sch_emp, connection)


In [261]:
vacancies_sch_emp

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


In [262]:
# Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.
query_vacancies_experience = f'''select 
                                v.experience,
                                COUNT(v.id)                                
                            from vacancies v
                            GROUP BY 1
                            ORDER BY 2
                            LIMIT 4
                         '''
vacancies_experience = pd.read_sql_query(query_vacancies_experience, connection)
vacancies_experience


C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\418698386.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_experience = pd.read_sql_query(query_vacancies_experience, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


# Вывод
По детальному анализу вакансий.
Распределение вакансий по регионам - в первую очередь это крупнейшие города, а также столицы и крупные города стран СНГ.
Нижняя средняя граница запплаты составляет 70 т.р., верхняя средняя примерно 110 т.р.
В основном требуется полная занятость на полный день, и опыт работы от 1 года до 3 лет.

# 5. Анализ работодателей

In [263]:
# Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.
query_ = f'''select 
                                e.name,
                                COUNT(v.id)                                
                            from vacancies v
                                JOIN employers e ON v.employer_id = e.id
                            GROUP BY 1
                            ORDER BY 2 DESC
                            LIMIT 5
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\2915165933.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


In [264]:
# Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
# Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.
query_ = f'''select 
                                a.name,
                                COUNT(e.id),
                                COUNT(v.id)                                
                            from areas a
                                LEFT JOIN vacancies v ON v.area_id = a.id
                                    LEFT JOIN employers e ON e.area = a.id
                            GROUP BY 1
                            HAVING (COUNT(v.id) = 0)
                            ORDER BY 2 DESC
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\810813858.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,name,count,count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
588,Струнино,0,0
589,Славгород (Алтайский край),0,0
590,Сорочинск,0,0
591,Ленинский (Тульская область),0,0


In [265]:
areas_df.head(1)

,id,name
0,2758,Тбилиси


In [266]:
vacancies_df.head(1)

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55312386,Компьютерный Мастер,Пользователь ПК\tРабота в команде\tРемонт ноут...,Полный день,Нет опыта,Полная занятость,64000.0,NaN,1516,5724811


In [267]:
# Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.
# Выберите максимальное значение из получившегося списка.
query_ = f'''select 
                                v.employer_id,
                                COUNT(DISTINCT v.area_id)
                            from vacancies v
                            GROUP BY 1
                            ORDER BY 2 DESC
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\2619357426.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,employer_id,count
0,1740,181
1,2748,152
2,5724811,116
3,5130287,88
4,3682876,71
...,...,...
14901,810278,1
14902,810313,1
14903,810551,1
14904,810688,1


In [268]:
# Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.
# Введите количество, которое вернул запрос, в поле ниже.
query_ = f'''select 
                                e.id,
                                COUNT(ei.employer_id)
                            from employers e
                                LEFT JOIN employers_industries ei ON e.id = ei.employer_id
                            GROUP BY 1
                            HAVING COUNT(ei.industry_id)=0
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_


C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\2163463973.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,id,count
0,4898037,0
1,2056231,0
2,4808196,0
3,1166354,0
4,180028,0
...,...,...
8414,5849807,0
8415,2276466,0
8416,2351276,0
8417,880080,0


In [279]:
ind_df

,id,name
0,7.540,Разработка программного обеспечения
1,7.539,"Системная интеграция, автоматизации технологи..."
2,27.550,Безалкогольные напитки (производство)
3,27.551,"Безалкогольные напитки (продвижение, оптовая т..."
4,13.664,Управление и эксплуатация недвижимости
...,...,...
289,37.626,"Ассоциация в сфере культуры, искусства"
290,42.603,Табак (производство)
291,29.532,"Рыболовство, рыбоводство"
292,37.627,Благотворительная организация


In [289]:
# Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.
# Введите в поле ниже название этой компании так же, как оно указано в результате запроса.
query_ = f'''select 
                                e.name
                            FROM
                                employers_industries ei
                                LEFT JOIN employers e ON ei.employer_id = e.id 
                            GROUP BY e.id 
                            HAVING COUNT(ei.industry_id) = 4 
                            ORDER BY name
                            OFFSET 2 -- отступ на 2 строки
                            LIMIT 1 
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\57350848.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,name
0,2ГИС


In [290]:
# С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».
query_ = f'''select 
                                COUNT(*)
                            from employers_industries ei
                                JOIN industries i ON ei.industry_id = i.id
                            WHERE i.name='Разработка программного обеспечения'
                            ORDER BY 1
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\4161080529.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,count
0,3553


Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. 
Также добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице.
Должна получиться выборка такого вида (приведён пример результата для компании SberTech):

In [ ]:
city_list = ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград']
city_list = tuple(city_list)
city_list 

In [ ]:
query_ = f'''WITH tab AS (select 
                                a.name,
                                COUNT(v.id)::int cnt 
                            from vacancies v
                                JOIN areas a ON v.area_id = a.id
                                    JOIN employers e ON v.employer_id = e.id
                            WHERE e.name='Яндекс' AND a.name IN {city_list }
                            GROUP BY 1
                            ORDER BY 2 DESC)
                            
                        SELECT 
                            *
                        FROM
                            tab
                        UNION -- добавляем к имеющейся таблице tab общее количество вакансий по всем городам-миллионникам из нее же
                        SELECT
                            'Total',
                            SUM(cnt) -- общая сумма вакансий
                        FROM
                            tab
                        ORDER BY cnt -- ну и сортируем по количеству вакансий по возрастанию
                         '''


In [ ]:
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\58841962.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


In [ ]:
print(ei_df.info())

# Вывод.
Максимальное количество вакансий составляет 1933 у компании Яндекс, 4 % всех вакансий.
На первом и пятом месте по количеству вакансий: Яндекс и Газпромнефть.
186 регионов без вакансий.
Яндекс на первом месте c результатом 181 регион. 
Почти 8.5 тыс. работодателей не указали сферы деятельности в которых они предлагают работу. У 2ГИС 4 сферы деятельности.
У 3553 работодателей указано в качестве сферы деятельности указана "Разработка программного обеспечения".
Количество вакансий Яндекса по городам-миллионникам, которых на момент было 16. Всего 485 вакансий, что составляет четверть от всех вакансий Яндекса.

# 6. Предметный анализ

In [283]:
# Сколько вакансий имеет отношение к данным (название содержит 'data' or ''данн)
# WHERE v.name ILIKE '%data%' OR v.name LIKE '%данн%'
query_ = f'''select 
                                v.name
                            from vacancies v
                            WHERE (lower(v.name) LIKE '%data%' OR lower(v.name) LIKE '%данн%')
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\1041755213.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,name
0,Аналитик хранилищ данных и BI-систем/Аналитик ...
1,Data-analysts (удаленно)
2,Стажер Data Engineer/Data Аналитик в команду Б...
3,Старший специалист хранилищ данных
4,Администратор баз данных и сетей
...,...
1766,Программист Delphi / разработчик баз данных
1767,Разработчик баз данных (Big Data)
1768,Аналитик данных
1769,Data аналитик /Аналитик BI (удаленно)


In [ ]:
# Сколько есть подходящих вакансий для начинающего дата-сайентиста?
# Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:

#‘data scientist’;
#‘data science’;
#‘исследователь данных’;
#‘ML’ (здесь не нужно брать вакансии по HTML);
#‘machine learning’;
#‘машинн%обучен%’.
#В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).

#Считаем вакансиями для специалистов уровня Junior следующие:

#в названии есть слово “junior” или
#требуемый опыт — «Нет опыта» или
#тип трудоустройства — «Стажировка».


In [284]:
query_ = f'''select 
                               Count(v.name)
                            from vacancies v
                            WHERE ((lower(v.name) LIKE '%data scientist%' OR lower(v.name) LIKE '%data science%' OR lower(v.name) LIKE '%исследователь данных%'
                                   OR lower(v.name) LIKE '%ml%' OR lower(v.name) LIKE '%machine learning%' OR lower(v.name) LIKE '%машинн%обучен%')
                                   AND (lower(v.name) LIKE '%junior%' OR lower(v.experience) LIKE '%нет опыта%' OR lower(v.employment) LIKE '%стажировка%')
                                   AND lower(v.name) NOT LIKE '%html%')
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\2962458856.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,count
0,51


In [ ]:
# Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?
query_ = f'''select 
                               COUNT(*)
                            from vacancies v
                            WHERE  (lower(v.name) LIKE '%data scientist%' OR 
                                    lower(v.name) LIKE '%data science%' OR
                                    lower(v.name) LIKE '%исследователь данных%' OR
                                   (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%') OR
                                    lower(v.name) LIKE '%machine learning%' OR
                                    lower(v.name) LIKE '%машинн%обучен%')
                                    AND
                                    (v.key_skills LIKE '%SQL%' OR
                                    lower(v.key_skills) LIKE '%postgres%')
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\3292588666.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,count
0,201


In [285]:
# С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. 
# Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.
query_ = f'''select 
                               COUNT(*)
                            from vacancies v
                            WHERE  (lower(v.name) LIKE '%data scientist%' OR 
                                    lower(v.name) LIKE '%data science%' OR
                                    lower(v.name) LIKE '%исследователь данных%' OR
                                   (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%') OR
                                    lower(v.name) LIKE '%machine learning%' OR
                                    lower(v.name) LIKE '%машинн%обучен%')
                                    AND
                                    (v.key_skills LIKE '%Python%')
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\2395140900.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,count
0,351


In [286]:
# Сколько ключевых навыков в среднем указывают в вакансиях для DS?
query_ = f'''select 
                               ROUND(AVG(length(v.key_skills) - length(replace(v.key_skills,'\t','')) + 1),2)
                            from vacancies v
                            WHERE   lower(v.name) LIKE '%data scientist%' OR 
                                    lower(v.name) LIKE '%data science%' OR
                                    lower(v.name) LIKE '%исследователь данных%' OR
                                   (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%') OR
                                    lower(v.name) LIKE '%machine learning%' OR
                                    lower(v.name) LIKE '%машинн%обучен%'
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\754663415.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,round
0,6.41


Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).
При решении задачи примите во внимание следующее:

Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
Если заполнены оба поля с зарплатой, считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, его и считаем зарплатой по вакансии.
Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null).
Чтобы избежать этой ситуации, мы воспользуемся функцией coalesce , которая заменит null на значение, которое мы передадим. 
Например, посмотрите, что возвращает запрос select 1 + coalesce(null, 0).

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет.

In [ ]:
query_ = f'''select 
                               v.experience,
                               ROUND(AVG((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from))/2))
                            from vacancies v
                            WHERE   (lower(v.name) LIKE '%data scientist%' OR 
                                    lower(v.name) LIKE '%data science%' OR
                                    lower(v.name) LIKE '%исследователь данных%' OR
                                   (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%') OR
                                    lower(v.name) LIKE '%machine learning%' OR
                                    lower(v.name) LIKE '%машинн%обучен%')
                                    AND
                                    (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
                                GROUP BY 1
                         '''
answer_ = pd.read_sql_query(query_, connection)
answer_

C:\Users\MCI\AppData\Local\Temp\ipykernel_4668\3624070404.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_ = pd.read_sql_query(query_, connection)


,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


# Вывод:
Для начинающего дата-сайентиста есть 51 вакансия.
Если владеть SQL или postgres можно рассматривать 201 вакансию.
Python у соискателей работодатели в 351 вакансии.
Зарплата. Чем больше опыт - тем больше ЗП. Сотрудникам без опыта 75 тыс. рублей. С опытом от года и до 3 лет - 140 тыс. руб. А от 3 и до 6 лет: 243 тыс. руб. 
в целом по базе данных зарплата (минимум, максимум или обе границы) указаны в 49 % случаев. 
Макс и мин ЗП в вакансиях: 55.5 тыс. руб. и 550 тыс.

In [ ]:
# не забываем закрыть соединение после окончания работы
connection.close()